# Quantum Approximate Optimization Algorithm (QAOA)

With one layer, p=1

In [12]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator,Aer
from qiskit.quantum_info import Statevector, SparsePauliOp
from scipy.optimize import minimize

# Define graph edges (square with one diagonal)
edges = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]
n_qubits = 4

# Define cost Hamiltonian H_C = sum (1 - Z_i Z_j)/2
pauli_list = []
for (i, j) in edges:
    z_str = ['I'] * n_qubits
    z_str[i] = 'Z'
    z_str[j] = 'Z'
    pauli_list.append((''.join(reversed(z_str)), -0.5))  # coefficient for -1/2 Z_i Z_j
# Constant term (1/2 per edge)
pauli_list.append(('I'*n_qubits, 0.5 * len(edges)))
H_C = SparsePauliOp.from_list(pauli_list)

# Backend for statevector simulation
sim = Aer.get_backend('statevector_simulator')

# Function to build and evaluate the QAOA circuit for given (gamma, beta)
def qaoa_expectation(params):
    gamma, beta = params
    qc = QuantumCircuit(n_qubits)
    
    # Step 1: Initialize |+>^n
    for q in range(n_qubits):
        qc.h(q)
    
    # Step 2: Apply cost unitary e^{-i γ H_C}
    for (i, j) in edges:
        qc.rzz(-2* gamma,i,j)
        #qc.cx(i, j)
        #qc.rz(-2 * gamma, j)
        #qc.cx(i, j)
    
    # Step 3: Apply mixer unitary e^{-i β H_M}
    for q in range(n_qubits):
        qc.rx(2 * beta, q)

    # Simulate circuit
    result = sim.run(qc).result()
    psi = result.get_statevector(qc)  
    
    # Compute expectation value
    exp_val = np.real(psi.expectation_value(H_C))
    
    # Return negative value (since we want to maximize)
    return -exp_val

# Initial guess for parameters
init_params = [0.5, 0.5]

# Run classical optimization
res = minimize(qaoa_expectation, init_params, method='Nelder-Mead',
               options={'maxiter': 100, 'disp': True})

# Print results
opt_gamma, opt_beta = res.x
max_exp_val = -res.fun

print("\n=== Optimization Results ===")
print(f"Optimal γ = {opt_gamma:.4f}")
print(f"Optimal β = {opt_beta:.4f}")
print(f"Maximum Expectation Value ⟨H_C⟩ = {max_exp_val:.4f}")



Optimization terminated successfully.
         Current function value: -3.237109
         Iterations: 35
         Function evaluations: 67

=== Optimization Results ===
Optimal γ = 0.2856
Optimal β = 0.3109
Maximum Expectation Value ⟨H_C⟩ = 3.2371


Now getting results in bit string manner

In [13]:
from qiskit.visualization import plot_histogram

#Building optimized circuit
qc_opt = QuantumCircuit(n_qubits)
for q in range(n_qubits):
    qc_opt.h(q)
for (i, j) in edges:
    qc_opt.rzz(-2 * opt_gamma, i, j)
for q in range(n_qubits):
    qc_opt.rx(2 * opt_beta, q)
qc_opt.measure_all()

sim_shot = AerSimulator()
qc_opt = qc_opt.copy()
qc_opt.measure_all()

# Run the circuit with finite shots
result = sim_shot.run(qc_opt, shots=2000).result()
counts = result.get_counts(qc_opt)


plot_histogram(counts)

def cut_value(bitstring, edges):
    """Compute cut value for given bitstring."""
    return sum(bitstring[i] != bitstring[j] for (i, j) in edges)

# Example: find the best bitstring from results
best_bitstring = max(counts, key=counts.get)
best_cut = cut_value(best_bitstring[::-1], edges)  # reverse since Qiskit bit order
print(f"\nBest bitstring = {best_bitstring}, Cut value = {best_cut}")




Best bitstring = 1010 1010, Cut value = 4


In [8]:

import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator,Aer
from qiskit.quantum_info import Statevector, Pauli
from qiskit import transpile
from scipy.optimize import minimize
import math
import random

# ---------- Utility functions ----------

def build_cost_layer(qc: QuantumCircuit, edges, gamma):
    for (i, j) in edges:
        qc.rzz(- 2 * gamma,i,j)
        
def build_mixer_layer(qc: QuantumCircuit, n_qubits, beta):
    angle = 2 * beta
    for q in range(n_qubits):
        qc.rx(angle, q)

def qaoa_statevector(params, p, n_qubits, edges):
    gammas = params[:p]
    betas = params[p:]
    qc = QuantumCircuit(n_qubits)

    # initial |+>^n
    for q in range(n_qubits):
        qc.h(q)

    # p layers
    for l in range(p):
        build_cost_layer(qc, edges, gammas[l])
        build_mixer_layer(qc, n_qubits, betas[l])

    # simulate
    sim = Aer.get_backend('statevector_simulator')
    # get statevector
    result = sim.run(qc).result()
    sv = result.get_statevector(qc).data   #complex numpy array size 2^n
    return sv

def expectation_Hc_from_statevector(sv, n_qubits, edges):
    # Precompute expectation values of Z_i Z_j by diagonal basis evaluation
    # For computational-basis diagonal operators, we can evaluate directly from probabilities and bit parity.
    probs = np.abs(sv)**2
    N = len(probs)
    exp_sum = 0.0
    for (i, j) in edges:
        # compute <Z_i Z_j> = sum_z probs[z] * z_i * z_j where z_k in {+1,-1}
        val = 0.0
        for idx in range(N):
            # bit of qubit k in computational basis: (idx >> k) & 1
            bi = 1 - 2 * ((idx >> i) & 1)  # +1 if bit 0, -1 if bit 1
            bj = 1 - 2 * ((idx >> j) & 1)
            val += probs[idx] * (bi * bj)
        exp_sum += 0.5 * (1.0 - val)
    return exp_sum

def maxcut_value_from_bitstring(bitstr, edges):
    """bitstr: list or array of 0/1 ints"""
    cut = 0
    for (i, j) in edges:
        if bitstr[i] != bitstr[j]:
            cut += 1
    return cut

# ---------- Main QAOA routine ----------

def qaoa_optimize(adj_matrix, p=1, method='COBYLA', initial_point=None, shots_for_sampling=1024):
    """
    Run QAOA from scratch for given adjacency matrix (numpy array), depth p.
    Returns optimizer result and best measured bitstrings.
    """
    n = adj_matrix.shape[0]
    # build edge list (undirected, i<j)
    edges = [(i, j) for i in range(n) for j in range(i+1, n) if adj_matrix[i, j] != 0]

    # cost function to minimize (we minimize negative expectation to maximize cut)
    def objective(x):
        # x length should be 2*p: [gammas..., betas...]
        sv = qaoa_statevector(x, p, n, edges)
        exp = expectation_Hc_from_statevector(sv, n, edges)
        # we want to maximize exp, so return -exp
        return -exp

    # initial params
    if initial_point is None:
        rng = np.random.default_rng(42)
        init_gammas = rng.uniform(0, np.pi, size=p)
        init_betas = rng.uniform(0, np.pi/2, size=p)
        x0 = np.concatenate([init_gammas, init_betas])
    else:
        x0 = np.array(initial_point, dtype=float)

    # run classical optimizer
    print("Starting classical optimization (this may take a bit)...")
    res = minimize(objective, x0, method=method, options={'maxiter': 200})
    print("Optimization finished. success:", res.success, "message:", res.message)
    opt_params = res.x
    print("Optimal params (gammas, betas):", opt_params)

    # build final statevector and sample to get candidate bitstrings
    sv = qaoa_statevector(opt_params, p, n, edges)
    probs = np.abs(sv)**2
    # sample bitstrings according to probs
    Nshots = shots_for_sampling
    rng = np.random.default_rng(123)
    outcomes = rng.choice(len(probs), size=Nshots, p=probs)
    counts = {}
    for out in outcomes:
        bstr = format(out, '0{}b'.format(n))[::-1]  # qiskit little-endian ordering used above
        # reverse or not: our expectation calculation used idx bit positions such that bit k = (idx >> k) & 1,
        # so we keep this ordering (LSB is qubit 0).
        counts[bstr] = counts.get(bstr, 0) + 1

    # evaluate cut values for unique outcomes
    best = {'bitstring': None, 'cut': -1, 'count': 0}
    detailed = []
    for bstr, cnt in counts.items():
        bits = [int(ch) for ch in bstr]
        cutval = maxcut_value_from_bitstring(bits, edges)
        detailed.append((bstr, cnt, cutval))
        if cutval > best['cut'] or (cutval == best['cut'] and cnt > best['count']):
            best.update({'bitstring': bstr, 'cut': cutval, 'count': cnt})

    return {
        'res': res,
        'optimal_params': opt_params,
        'statevector': sv,
        'probabilities': probs,
        'samples': counts,
        'detailed': detailed,
        'best': best,
        'edges': edges
    }

# ---------- Example usage for a 4-vertex square graph ----------
if __name__ == "__main__":
    # Define adjacency for 4-vertex square (0-1-2-3-0)
    n = 5
    A = np.zeros((n, n), dtype=int)
    edges = [(0,1),(1,2),(2,3),(3,4),(4,0),(0,3),(0,2),(4,2)]
    for (i,j) in edges:
        A[i,j] = 1
        A[j,i] = 1

    # Run QAOA from scratch with p=1
    p = 1
    result = qaoa_optimize(A, p=p, method='COBYLA', shots_for_sampling=2000)

    # Print summary
    print("\nEdges:", result['edges'])
    print("Best measured bitstring (LSB=qubit0):", result['best']['bitstring'])
    print("Best cut value observed:", result['best']['cut'], " (count:", result['best']['count'], ")")
    print("\nTop measured outcomes (bitstring, counts, cut):")
    # sort by cut desc then count
    for bstr, cnt, cut in sorted(result['detailed'], key=lambda x: (-x[2], -x[1]))[:10]:
        print(bstr, cnt, cut)


Starting classical optimization (this may take a bit)...
Optimization finished. success: True message: Return from COBYLA because the trust region radius reaches its lower bound.
Optimal params (gammas, betas): [1.81722614 1.98681814]

Edges: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (2, 3), (2, 4), (3, 4)]
Best measured bitstring (LSB=qubit0): 10100
Best cut value observed: 6  (count: 184 )

Top measured outcomes (bitstring, counts, cut):
10100 184 6
01011 166 6
10110 63 5
01010 63 5
10101 52 5
01001 50 5
11001 35 5
10001 28 5
00101 25 5
01101 25 5
